In [479]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [480]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [481]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [482]:
%%capture pwd
!pwd

In [483]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('us-inflation-is-it-back-to-the-future', 300, 500)

# Fig 1

In [484]:
df = pd.read_excel("raw/new/Chart1 Long-term Inflation.xls", skiprows=11)
df

,Date,CPI,Recession
0,1948-01-01,10.24209,-1
1,1948-02-01,9.48196,-1
2,1948-03-01,6.81818,-1
3,1948-04-01,8.27273,-1
4,1948-05-01,9.38497,-1
...,...,...,...
883,2021-08-01,5.20248,-1
884,2021-09-01,5.37730,-1
885,2021-10-01,6.24352,-1
886,2021-11-01,6.88047,-1


In [485]:
f = "fig1_cpi"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Date,CPI,Recession
0,1948-01-01,10.24209,-1
1,1948-02-01,9.48196,-1
2,1948-03-01,6.81818,-1
3,1948-04-01,8.27273,-1
4,1948-05-01,9.38497,-1


In [486]:
recess = (
    df.loc[df["Recession"].diff().replace(0, np.nan).dropna().index]["Date"]
    .astype(str)
    .values
)
recessions = pd.DataFrame(
    list(
        np.array(
            [
                [
                    {"date": recess[2 * i], "y": -4, "y2": 16, "c": i},
                    {"date": recess[2 * i + 1], "y": -4, "y2": 16, "c": i},
                ]
                for i in range(len(recess[::2]))
            ]
        ).flatten()
    )
)

In [487]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "Date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        line={"color": colors["eco-turquiose"]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
                alt.GradientStop(color="white", offset=0.2),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        y=alt.Y(
            "CPI:Q",
            sort=[],
            axis=alt.Axis(
                gridColor=colors["eco-gray"],
                gridOpacity=0.07,
                title="US CPI inflation % YoY",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[-4, 16]),
        ),
    )
    .transform_calculate(y2="-4")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y="y:Q")
)
bands = (
    alt.Chart(recessions)
    .mark_area(opacity=0.2)
    .encode(
        x="date:T",
        y="y:Q",
        y2="y2:Q",
        color=alt.Color(
            "c:N", legend=None, scale=alt.Scale(range=[colors["eco-gray"]])
        ),
    )
)
labels = (
    alt.Chart(pd.DataFrame([{"y": 15.2, "x": "1991-06-01", "t": "Recessions"}]))
    .mark_text(align="left", opacity=0.5, color=colors["eco-gray"])
    .encode(y="y:Q", x="x:T", text="t:N")
)
layer1 = (
    ((bands + zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [488]:
df = pd.read_excel("raw/new/core inflation measures.xlsx", skiprows=432, usecols="I:M")
df["Date"] = pd.to_datetime(df["Date "]).astype(str)
df = df.drop("Date ", axis=1).set_index("Date").stack().reset_index()
df.columns = ["date", "indicator", "value"]

In [489]:
f = "fig2_core_inflation_measures"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,indicator,value
0,2019-01-01,16% Trimmed Mean,2.203080
1,2019-01-01,Core,2.117647
2,2019-01-01,Median,2.660832
3,2019-01-01,Headline,1.487736
4,2019-02-01,16% Trimmed Mean,2.175578


In [490]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line().encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="Core inflation measures % YoY",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "indicator:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-dot"],
                colors["eco-green"],
                colors["eco-mid-blue"],
            ]
        ),
    ),
)
labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="indicator:N")
    .transform_filter("year(datum.date)==2021")
    .transform_filter("month(datum.date)==11")
)
layer1 = (
    ((line + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [491]:
df = pd.read_excel("raw/copr pce-core cpi.xls", skiprows=10).drop("Date", axis=1)
df.columns = ["value", "date"]

In [492]:
f = "fig3_pce_core_cpi"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,value,date
0,-0.55895,2001-01-01
1,-0.75227,2001-02-01
2,-0.67888,2001-03-01
3,-0.63541,2001-04-01
4,-0.60434,2001-05-01


In [493]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = (
    base.mark_area(
        interpolate="monotone",
        fillOpacity=0.7,
        line={"color": colors["eco-turquiose"]},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color=colors["eco-turquiose"], offset=0.7),
                alt.GradientStop(color="white", offset=0.2),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="Core PCE - Core CPI gap in % points",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                tickCount=4,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=[-1.6, 1]),
        ),
    )
    .transform_calculate(y2="-4")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y="y:Q")
)
bands = (
    alt.Chart(
        recessions[pd.to_datetime(recessions["date"]) > pd.to_datetime("2000-10-01")]
        .replace(-4, -1.6)
        .replace(16, 1)
    )
    .mark_area(opacity=0.2)
    .encode(
        x="date:T",
        y="y:Q",
        y2="y2:Q",
        color=alt.Color(
            "c:N", legend=None, scale=alt.Scale(range=[colors["eco-gray"]])
        ),
    )
)
labels = (
    alt.Chart(pd.DataFrame([{"y": 0.9, "x": "2002-02-01", "t": "Recessions"}]))
    .mark_text(align="left", opacity=0.5, color=colors["eco-gray"])
    .encode(y="y:Q", x="x:T", text="t:N")
)
layer1 = (
    ((bands + zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 4

In [494]:
df = pd.read_csv("raw/new/consumption_durables_nond_services.csv", header=None)[
    [0, 1, 2, 3]
]
df.columns = df.loc[5].values
df = df.replace("Durables", np.nan).dropna()
df["DATE"] = pd.to_datetime(df["DATE"]).astype(str)
df = df.set_index("DATE").stack().reset_index()
df.columns = ["date", "durables", "value"]

In [495]:
f = "fig4_consumption patterns"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,durables,value
0,2019-01-01,Non-durables,98.15406
1,2019-01-01,Durables,94.98897
2,2019-01-01,Services,98.29339
3,2019-02-01,Non-durables,97.73498
4,2019-02-01,Durables,94.91107


In [496]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line().encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="Indexed to December 2019 = 100",
            titleX=5,
            titleY=35,
            titleBaseline="bottom",
            titleAngle=0,
            tickCount=6,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "durables:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-mid-blue"],
                colors["eco-green"],
                colors["eco-mid-blue"],
            ]
        ),
    ),
)

labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="durables:N")
    .transform_filter("year(datum.date)==2021")
    .transform_filter("month(datum.date)==9")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 100}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q", scale=alt.Scale(domain=[60, 150])))
)
layer1 = (
    ((zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 5

In [497]:
df = pd.read_csv("raw/new/PCEinflationUSE1.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True).astype(str)
df = df.set_index("Date").stack().reset_index()
df.columns = ["date", "durables", "value"]
df = df.replace("Nondurables", "Non-durables")

In [498]:
f = "fig5_pce_inflation"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,durables,value
0,1990-01-01,Durables,0.83225
1,1990-01-01,Services,5.61139
2,1990-01-01,Non-durables,4.31248
3,1990-02-01,Durables,1.20998
4,1990-02-01,Services,5.88427


In [499]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="% YoY",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                tickCount=6,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        color=alt.Color(
            "durables:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter("year(datum.date)>=2015")
)

labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="durables:N")
    .transform_filter("year(datum.date)==2021")
    .transform_filter("month(datum.date)==10")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q"))
)
layer1 = (
    ((zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 6

In [500]:
df = pd.read_excel("raw/new/OER and shiller.xls", skiprows=180, usecols="G:J")
df = df.set_index("Date").stack().reset_index()
df.columns = ["date", "indicator", "value"]

In [501]:
f = "fig6_house_prices"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,indicator,value
0,2004-01-01,House Prices (Lagged),8.28095
1,2004-01-01,Owner's Equivalent Rent,1.87815
2,2004-01-01,Rent,2.45942
3,2004-02-01,House Prices (Lagged),8.47139
4,2004-02-01,Owner's Equivalent Rent,1.92132


In [502]:
base = alt.Chart(f6).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line().encode(
    color=alt.Color(
        "indicator:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-mid-blue"],
                colors["eco-gray"],
            ]
        ),
    ),
)
line1 = line.transform_filter("datum.indicator!='House Prices (Lagged)'").encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="% YoY",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            tickCount=6,
            titleAlign="left",
            labelColor=colors["eco-mid-blue"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-mid-blue"],
            domainColor=colors["eco-mid-blue"],
        ),
    )
)
line2 = line.transform_filter("datum.indicator=='House Prices (Lagged)'").encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="% YoY",
            titleX=-25,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            tickCount=6,
            titleAlign="left",
            labelColor=colors["eco-turquiose"],
            titleColor=colors["eco-turquiose"],
            tickColor=colors["eco-turquiose"],
            domainColor=colors["eco-turquiose"],
        ),
    )
)

labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"date": "2015-01-01", "y": -0.5, "t": " Owner's Equivalent Rent"},
                {"date": "2020-03-01", "y": 18, "t": " House Prices (Lagged)"},
                {"date": "2014-01-01", "y": 11, "t": " Rent"},
            ]
        )
    )
    .mark_text()
    .encode(y="y:Q", x="date:T", text="t:N", color="t:N")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q"))
)
layer1 = (
    (((line1) + (line2 + labels)).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
).resolve_scale(y="independent")
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 7

In [503]:
df1 = pd.read_csv("raw/new/7-1.csv").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("raw/new/7-2.csv").drop("Unnamed: 0", axis=1)
df = df1.set_index("Date").join(df2.set_index("Date")).stack().reset_index()
df.columns = ["date", "indicator", "value"]
df["indicator"] = df["indicator"].str.replace("_", " ")

In [504]:
f = "fig7_michigan"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,indicator,value
0,2021-12-31,Median 1 year,4.8
1,2021-12-31,Median 5 year,2.9
2,2021-11-30,Median 1 year,4.9
3,2021-11-30,Median 5 year,3.0
4,2021-10-31,Median 1 year,4.8


In [505]:
base = alt.Chart(f7).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="Expected inflatoin rate % YoY",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                tickCount=6,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        color=alt.Color(
            "indicator:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter("year(datum.date)>=2000")
)

labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="indicator:N")
    .transform_filter("year(datum.date)==2021")
    .transform_filter("month(datum.date)==10")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q"))
)
layer1 = (
    ((zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 8

In [506]:
df1 = pd.read_csv("raw/new/8-1.csv").drop("Unnamed: 0", axis=1)
df2 = pd.read_csv("raw/new/8-2.csv").drop("Unnamed: 0", axis=1)
df = df1.set_index("Date").join(df2.set_index("Date")).stack().reset_index()
df.columns = ["date", "indicator", "value"]
df["indicator"] = df["indicator"].str.replace("_", " ")

In [507]:
f = "fig8_michigan_sd"
f8 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f8.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f8 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,indicator,value
0,2021-12-31,SD 1 year,7.2
1,2021-12-31,SD 5 year,4.3
2,2021-11-30,SD 1 year,6.6
3,2021-11-30,SD 5 year,4.2
4,2021-10-31,SD 1 year,6.5


In [508]:
base = alt.Chart(f8).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = (
    base.mark_line()
    .encode(
        y=alt.Y(
            "value:Q",
            sort=[],
            axis=alt.Axis(
                gridColor=colors["eco-gray"],
                gridOpacity=0.1,
                title="Expected inflatoin rate % YoY",
                titleX=-5,
                titleY=-5,
                titleBaseline="bottom",
                titleAngle=0,
                tickCount=6,
                titleAlign="left",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        color=alt.Color(
            "indicator:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter("year(datum.date)>=1990")
)

labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="indicator:N")
    .transform_filter("year(datum.date)==2021")
    .transform_filter("month(datum.date)==10")
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y=alt.Y("y:Q"))
)
layer1 = (
    ((zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 9

In [509]:
df = (
    pd.read_excel("raw/new/Recession dates and balck and white unemployment.xlsx")
    .dropna(axis=1)
    .drop("Recession", axis=1)
)
df = df.set_index("Date").stack().reset_index()
df.columns = ["date", "indicator", "value"]

In [510]:
f = "fig9_racial"
f9 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f9.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f9 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,indicator,value
0,1980-01-01,Black,13.0
1,1980-01-01,White,5.5
2,1980-02-01,Black,12.9
3,1980-02-01,White,5.5
4,1980-03-01,Black,12.9


In [511]:
base = alt.Chart(f9).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line().encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.07,
            title="Unemployment %",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
        scale=alt.Scale(domain=[0, 24]),
    ),
    color=alt.Color("indicator:N"),
)
zeroline = (
    alt.Chart(pd.DataFrame([{"y": 0}]))
    .mark_rule(strokeWidth=1, strokeDash=[5, 5], opacity=0.4, color=colors["eco-gray"])
    .encode(y="y:Q")
)
bands = (
    alt.Chart(recessions.replace(-4, 0).replace(16, 24))
    .mark_area(opacity=0.2)
    .encode(
        x="date:T",
        y="y:Q",
        y2="y2:Q",
        color=alt.Color(
            "c:N",
            legend=None,
            scale=alt.Scale(
                range=[
                    colors["eco-turquiose"],
                    colors["eco-mid-blue"],
                    colors["eco-gray"],
                ]
            ),
        ),
    )
    .transform_filter("year(datum.date)>=1980")
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"y": 23, "x": "1991-06-01", "t": "Recessions"},
                {"y": 17, "x": "2012-06-01", "t": "Black"},
                {"y": 2.3, "x": "2016-06-01", "t": "White"},
            ]
        )
    )
    .mark_text(align="left")
    .encode(y="y:Q", x="x:T", text="t:N", color="x:N")
)
layer1 = (
    ((bands + zeroline + line + labels).properties(height=330, width=450))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)